In [2]:
import pandas as pd

In [3]:
import regex as re

In [4]:
from itertools import chain
from collections import Counter

In [5]:
import os
from itertools import chain

In [6]:
df = pd.read_json('simple_news_scraper/steel_news.json', encoding='utf-8')

In [7]:
df = df.explode(['TEXT']).reset_index(drop=True)

In [8]:
df.TEXT = df.TEXT.apply(lambda x: x.strip())

In [9]:
df = df.dropna().reset_index(drop=True)

In [10]:
dict_duplicated = dict(df.TEXT.value_counts().head(20))

In [11]:
dict_duplicated = {k:v for k,v in dict_duplicated.items() if v>100}

In [12]:
df = df[df.TEXT.apply(lambda x: x not in dict_duplicated)].reset_index(drop=True)

In [13]:
len(dict_duplicated)

19

In [14]:
dict_duplicated

{'[email\xa0protected]': 4912,
 'بريد الكتروني :': 2628,
 '': 2207,
 'Mail:': 2145,
 'مكتب القاهرة الاقليمي': 1317,
 'الاتحاد هو منظمة غير حكومية ذات طبيعة خاصة ، وليس لها طابع سياسي أو تجاري.': 1316,
 'هاتف +21323304221 فاكس +21323304254': 1316,
 'يعمل الاتحاد في مجال إعداد الدراسات وتنظيم الدورات وعقد المؤتمرات الدورية لازدهار صناعة الحديد والصلب العربية، ويضم الاتحاد شريحة واسعة من الشركات متعددة الأنشطة متعلقة بصناعة الحديد والصلب.': 1316,
 'هاتف +20233356219 فاكس +20233374790': 1316,
 'الامانة العامة': 1316,
 'تأسس الاتحاد العربي للحديد والصلب في الجزائر عام 1971 كأول اتحاد عربي للدول العربية يتم إنشاؤه تحت مظلة مجلس الوحدة الاقتصادية في جامعة الدول العربية.': 1316,
 'Cairo Regional Office': 1076,
 'Arab Iron and Steel Union ( AISU ) was established in Algeria in 1971 as the first Arab union of Arab countries to be established under the umbrella of the Council of Economic Unity in the League of Arab States.': 1075,
 'AISU is a non-governmental organization of a private nature, not

In [15]:
df

,URL,TEXT
0,https://aisusteel.org/en/12537/,The modified unaudited financial results of th...
1,https://aisusteel.org/en/12537/,Sales slumped to EGP 1.08 billion in the year ...
2,https://aisusteel.org/en/12537/,"During the first nine months of FY19/20, the c..."
3,https://aisusteel.org/en/6518/,"Nucor Steel has awarded SMS group, Inc. the co..."
4,https://aisusteel.org/en/6518/,The scope includes two 190 metric ton direct c...
...,...,...
13429,https://aisusteel.org/en/9028/,"Meanwhile, in the first five months of this ye..."
13430,https://aisusteel.org/en/9028/,"In the given period, Russia ranked first among..."
13431,https://aisusteel.org/9745/,تمكن منتجو حديد التسليح في تركيا من زيادة صادر...
13432,https://aisusteel.org/9745/,وخلال الفترة من يناير إلى أغسطس ازدات صادرات ت...


In [16]:
import fasttext
model_loc = '../DATASETS/Fasttext_Languate_Detection/lid.176.bin'
model = fasttext.load_model(model_loc)

In [17]:
import regex as re

In [18]:
def detect_lang(text):
    fasttext_object = model.predict(text)
    lang = fasttext_object[0]
    score = fasttext_object[1]
    
    return (lang[0], score[0])

In [19]:
df.TEXT = df['TEXT'].astype(str)

In [20]:
df.TEXT = df.TEXT.apply(lambda x: re.sub(r'\s+', ' ', x))

In [21]:
df = df[df.TEXT.apply(lambda x: len(x.split())>1)].reset_index(drop=True)

In [22]:
df['LANG'] = df['TEXT'].apply(lambda x: detect_lang(x))

In [23]:
df['LANG_DETECT'] = df['LANG'].apply(lambda x : x[0])
df['LANG_SCORE'] = df['LANG'].apply(lambda x : x[1])

In [24]:
df_en = df[df.LANG_DETECT == '__label__en'].reset_index(drop=True)

In [25]:
df_en

,URL,TEXT,LANG,LANG_DETECT,LANG_SCORE
0,https://aisusteel.org/en/12537/,The modified unaudited financial results of th...,"(__label__en, 0.943916380405426)",__label__en,0.943916
1,https://aisusteel.org/en/12537/,Sales slumped to EGP 1.08 billion in the year ...,"(__label__en, 0.964730978012085)",__label__en,0.964731
2,https://aisusteel.org/en/12537/,"During the first nine months of FY19/20, the c...","(__label__en, 0.9291689991950989)",__label__en,0.929169
3,https://aisusteel.org/en/6518/,"Nucor Steel has awarded SMS group, Inc. the co...","(__label__en, 0.9582651257514954)",__label__en,0.958265
4,https://aisusteel.org/en/6518/,The scope includes two 190 metric ton direct c...,"(__label__en, 0.8953480124473572)",__label__en,0.895348
...,...,...,...,...,...
5186,https://aisusteel.org/en/10150/,Arab countries’ production of crude steel,"(__label__en, 0.6892220973968506)",__label__en,0.689222
5187,https://aisusteel.org/en/9028/,According to the data provided by the Turkish ...,"(__label__en, 0.9031760096549988)",__label__en,0.903176
5188,https://aisusteel.org/en/9028/,The revenue from these imports amounted to $41...,"(__label__en, 0.9179209470748901)",__label__en,0.917921
5189,https://aisusteel.org/en/9028/,"Meanwhile, in the first five months of this ye...","(__label__en, 0.8969374895095825)",__label__en,0.896937


In [26]:
df_ar = df[df.LANG_DETECT == '__label__ar'].reset_index(drop=True)

In [27]:
df_ar

,URL,TEXT,LANG,LANG_DETECT,LANG_SCORE
0,https://aisusteel.org/8328/,شهد قطاع الصلب في الصين توسعا في الإنتاج في ال...,"(__label__ar, 0.9988604784011841)",__label__ar,0.998860
1,https://aisusteel.org/8328/,وفي الفترة من يناير إلى مارس، ارتفع إنتاج الصل...,"(__label__ar, 0.99996018409729)",__label__ar,0.999960
2,https://aisusteel.org/8328/,وذكرت الجمعية، نقلا عن بيانات من الهيئة الوطني...,"(__label__ar, 0.9993679523468018)",__label__ar,0.999368
3,https://aisusteel.org/8328/,وأظهرت البيانات أن إنتاج الحديد الخام ارتفع 8 ...,"(__label__ar, 0.9998103976249695)",__label__ar,0.999810
4,https://aisusteel.org/8328/,وقالت الجمعية إنه مع استمرار تعافي الصناعات ال...,"(__label__ar, 0.9983462691307068)",__label__ar,0.998346
...,...,...,...,...,...
6735,https://aisusteel.org/10037/,في إطار الجهود المبذولة لتعزيز التعاون بين الج...,"(__label__ar, 0.9948592185974121)",__label__ar,0.994859
6736,https://aisusteel.org/10037/,أقيمت الاحتفالية بالمقر الرئيسي لمجمع ايميتال ...,"(__label__ar, 0.9952947497367859)",__label__ar,0.995295
6737,https://aisusteel.org/9745/,تمكن منتجو حديد التسليح في تركيا من زيادة صادر...,"(__label__ar, 0.9994086027145386)",__label__ar,0.999409
6738,https://aisusteel.org/9745/,وخلال الفترة من يناير إلى أغسطس ازدات صادرات ت...,"(__label__ar, 0.9992495179176331)",__label__ar,0.999250
